In [ ]:
from __future__ import print_function
from torch import nn, optim
import numpy as np
from random import randint
import copy                                                # to keep track of the evolution of the weights of the network
import os   
# from sklearn import manifold                               # for the t-SNE
# import matplotlib                                          # for the t-SNE
# from itertools import cycle, islice                        # for the t-SNE
from matplotlib import pyplot as plt 
from collections import OrderedDict
import pandas as pd
import datetime
import time
import torch                        
import torch.utils.data
from torch.nn import functional as F
# from torchvision import transforms
from datasets import YeastDataset # dataloader loading the yeast dataset in the correct format, replace it with one suitable for your problem
from data_elaboration_utilities import *
print("Libraries loaded")

In [ ]:
# define the arrays, the dictionary and the variables
repetitions = 50                                                                # number of random attempts

parameters = {"batch_size" : None, 
              "epochs" : None,                          
              "learning_rate" : None,
              "dataset_path" : None,
              "training_path" : None,
              "test_path" : None,
              # "num_workers" : 1, 
              # "pin_memory" : True,      
              "num_neurons" : None,               
              "num_neurons2" : None,
              "optimizer" : None,
              "dropout" : None,
              "loss": None,
              "gamma" : 0.1,
              "validation_loss" : None,
              "training_loss" : None,
              "experiment": None}

# all of the below parameters can be modified according to your preferences

# batch_size = [32, 64, 128]            # same for training validation and test set, values for first part of the training

batch_size = {"fluxes" : [32, 64, 128], "genes" : [32, 64, 128], "all" : [32, 64, 128], 
              "RFECV_genes" : [32, 64, 128], "RFECV_fluxes" : [32, 64, 128], "RFECV_all" : [32, 64, 128], 
              "RFECV_genes_p" : [32, 64, 128], "FCBF_fluxes_p" : [32, 64, 128], 
              "FCBF_fluxes" : [32, 64, 128], "FCBF_genes" : [32, 64, 128], "FCBF_all" : [32, 64, 128], 
              "FCBF_genes_p" : [32, 64, 128], "FCBF_all_p" : [32, 64, 128]}

# epochs = [400, 800, 1200, 1600, 2000, 2400]      # values for first part of the training

epochs = {"fluxes" : [400, 800, 1200, 1600, 2000, 2400], "genes" : [400, 800, 1200, 1600, 2000, 2400], 
          "all" : [400, 800, 1200, 1600, 2000, 2400], "RFECV_genes" : [400, 800, 1200, 1600, 2000, 2400], 
          "RFECV_fluxes" : [400, 800, 1200, 1600, 2000, 2400], "RFECV_all" : [400, 800, 1200, 1600, 2000, 2400], 
          "RFECV_genes_p" : [400, 800, 1200, 1600, 2000, 2400], "FCBF_fluxes_p" : [400, 800, 1200, 1600, 2000, 2400], 
          "FCBF_fluxes" : [400, 800, 1200, 1600, 2000, 2400], "FCBF_genes" : [400, 800, 1200, 1600, 2000, 2400], 
          "FCBF_all" : [400, 800, 1200, 1600, 2000, 2400], "FCBF_genes_p" : [400, 800, 1200, 1600, 2000, 2400], 
          "FCBF_all_p" : [400, 800, 1200, 1600, 2000, 2400]}

# learning_rate =  [1e-2, 1e-3, 1e-4, 1e-5]        # values for first part of the training

learning_rate = {"fluxes" : [1e-2, 1e-3, 1e-4, 1e-5], "genes" : [1e-2, 1e-3, 1e-4, 1e-5], "all" : [1e-2, 1e-3, 1e-4, 1e-5], 
                 "RFECV_genes" : [1e-2, 1e-3, 1e-4, 1e-5], "RFECV_fluxes" : [1e-2, 1e-3, 1e-4, 1e-5], 
                 "RFECV_all" : [1e-2, 1e-3, 1e-4, 1e-5], "RFECV_genes_p" : [1e-2, 1e-3, 1e-4, 1e-5], 
                 "FCBF_fluxes_p" : [1e-2, 1e-3, 1e-4, 1e-5], "FCBF_fluxes" : [1e-2, 1e-3, 1e-4, 1e-5],
                 "FCBF_genes" : [1e-2, 1e-3, 1e-4, 1e-5], "FCBF_all" : [1e-2, 1e-3, 1e-4, 1e-5], 
                 "FCBF_genes_p" : [1e-2, 1e-3, 1e-4, 1e-5], "FCBF_all_p" : [1e-2, 1e-3, 1e-4, 1e-5]}

# dropout = [0, 0.3, 0.6]                          # values for first part of the training

dropout = {"fluxes" : [0, 0.3, 0.6], "genes" : [0, 0.3, 0.6], "all" : [0, 0.3, 0.6], 
           "RFECV_genes" : [0, 0.3, 0.6], "RFECV_fluxes" : [0, 0.3, 0.6], "RFECV_all" : [0, 0.3, 0.6], 
           "RFECV_genes_p" : [0, 0.3, 0.6], "FCBF_fluxes_p" : [0, 0.3, 0.6], 
           "FCBF_fluxes" : [0, 0.3, 0.6], "FCBF_genes" : [0, 0.3, 0.6], "FCBF_all" : [0, 0.3, 0.6], 
           "FCBF_genes_p" : [0, 0.3, 0.6], "FCBF_all_p" : [0, 0.3, 0.6]}

# optimizer = ["Adam", "SGD","Rprop", "Adadelta"]  # values for first part of the training

optimizer = {"fluxes" : ["Adam", "SGD","Rprop", "Adadelta"], "genes" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "all" : ["Adam", "SGD","Rprop", "Adadelta"], "RFECV_genes" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "RFECV_fluxes" : ["Adam", "SGD","Rprop", "Adadelta"], "RFECV_all" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "RFECV_genes_p" : ["Adam", "SGD","Rprop", "Adadelta"], "FCBF_fluxes_p" : ["Adam", "SGD","Rprop", "Adadelta"],
             "FCBF_fluxes" : ["Adam", "SGD","Rprop", "Adadelta"], "FCBF_genes" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "FCBF_all" : ["Adam", "SGD","Rprop", "Adadelta"], "FCBF_genes_p" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "FCBF_all_p" : ["Adam", "SGD","Rprop", "Adadelta"]}

# loss = ["L1_loss", "MSE_loss", "Smooth_l1_loss"]     # values for first part of the training

loss = {"fluxes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "genes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], 
        "all" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "RFECV_genes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], 
        "RFECV_fluxes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "RFECV_all" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], 
        "RFECV_genes_p" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "FCBF_fluxes_p" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"],
        "FCBF_fluxes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "FCBF_genes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], 
        "FCBF_all" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "FCBF_genes_p" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], 
             "FCBF_all_p" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"]}


paths = ["paths",  
         None # to distinguish between the two possible situations in which there is one or two datasets (not training, validation and test together)
        ] # add the paths to the csv containing your data according to the usage of the variable paths below (modifying the indices if necessary)

no_cuda = False               # SHOULD BE FALSE                    
seed = 1                   
log_interval = 10       
percent_train = 0.7  
percent_validation = 0.2

dataset_path = paths[-1] 
parameters["dataset_path"] = dataset_path

training_path = paths[18]    
validation_path = paths[19]
test_path = paths[20] 
parameters["training_path"] = training_path
parameters["validation_path"] = validation_path
parameters["test_path"] = test_path

input_size = 32

# change the values below according to the size of your dataset, the keys define the type of data
num_neurons = {"fluxes" : [900, 1200, 330, 200], "genes" : [10000, 15000, 3500, 2500], "all" : [12000, 17000, 4000, 2500], 
               "fluxes_p" : [220, 300, 50, 30], "genes_p" : [3300, 2700, 800, 650], "all_p" : [3300, 2800, 700, 450],
               "RFECV_genes" : [3400, 4100, 1100, 890], "RFECV_all" : [3200, 4000, 900, 750], "RFECV_fluxes" : [40, 52, 10, 7], 
               "RFECV_genes_p" : [390, 580, 130, 80], "FCBF_all" : [160, 230, 45, 33], "FCBF_genes" : [190, 270, 56, 42], "FCBF_fluxes" : [28, 34, 7, 5], 
               "FCBF_fluxes_p" : [9, 12, 3, 2], "FCBF_genes_p" : [55, 70, 20, 15], "FCBF_all_p" : [60, 80, 28, 18]}   
                                                    

num_neurons2 = {"fluxes" : [1500, 1800, 180, 40], "genes" : [20000, 28000, 4000, 5000], "all" : [18000, 24000, 5000, 1500], 
                "fluxes_p" : [300, 400, 20, 10], "genes_p" : [4200, 1600, 430, 250], "all_p" : [4200, 1600, 380, 180],
                "RFECV_fluxes" : [60, 70, 5, 3], "RFECV_genes" : [4590, 5460, 670, 500], "RFECV_all" : [4400, 5200, 570, 440], 
                "RFECV_genes_p" : [700, 900, 70, 30], "FCBF_all" : [290, 330, 20, 15], "FCBF_genes" : [340, 400, 34, 20], "FCBF_fluxes" : [48, 58, 4, 3], 
                "FCBF_fluxes_p" : [16, 22, 2, 1], "FCBF_genes_p" : [80, 110, 12, 8], "FCBF_all_p" : [110, 140, 15, 7]}

# define the mapping
mapping = {459 : "fluxes", 6170 : "genes", 6629 : "all", 1858 : "RFECV_genes", 19 : "RFECV_fluxes", 1664 : "RFECV_all",
           200 : "RFECV_genes_p", 4 : "FCBF_fluxes_p", 32 : "FCBF_genes_p", 36 : "FCBF_all_p",
           79 : "FCBF_genes", 12 : "FCBF_fluxes", 68 : "FCBF_all"}
num_n = mapping[input_size]

cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)

device = torch.device("cuda" if cuda else "cpu")    # use the GPU if available

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

if dataset_path != None:
    partition, labels = splitTrainingValidationTestSet(dataset_path, percent_train, percent_validation)
else:
    partition, labels = loadTrainingValidationTestSet(training_path, validation_path, test_path)
    
training_set = YeastDataset(partition['training'], labels)
validation_set = YeastDataset(partition['validation'], labels)
        
torch.set_printoptions(precision=9)                                 # to print more digits for the loss

print("Device: ", device)                           # simple check

In [ ]:
print(dataset_path)

In [5]:
def RandomIndexes():                                           # randomly choose the parameter
    int1 = randint(0, len(batch_size[num_n]) - 1)    
    int2 = randint(0, len(epochs[num_n]) - 1)
    int3 = randint(0, len(learning_rate[num_n]) - 1)
    int4 = randint(0, len(num_neurons[num_n]) - 1)                                  
    int5 = randint(0, len(num_neurons2[num_n]) - 1)
    int6 = randint(0, len(optimizer[num_n]) - 1)
    int7 = randint(0, len(dropout[num_n]) - 1)
    int8 = randint(0, len(loss[num_n]) - 1)                                        
    return int1, int2, int3, int4, int5, int6, int7, int8

def fillDict(d):
    ind1, ind2, ind3, ind4, ind5, ind6, ind7, ind8 = RandomIndexes()
    d["batch_size"] = batch_size[num_n][ind1]
    d["epochs"] = epochs[num_n][ind2]
    d["learning_rate"] = learning_rate[num_n][ind3]
    d["num_neurons"] = num_neurons[num_n][ind4]                          
    d["num_neurons2"] = num_neurons2[num_n][ind5]
    d["optimizer"] = optimizer[num_n][ind6]
    d["dropout"] = dropout[num_n][ind7]
    d["loss"] = loss[num_n][ind8]
    d = OrderedDict(sorted(d.items(), key=lambda t: t[0]))
    return d                                                                       

def checkIdentical(path, parameters):
    combinations = pd.read_csv(path) 
    if "Unnamed: 0" in combinations.columns:
        combinations.drop("Unnamed: 0", axis=1, inplace=True)               # change according to the structure of your file
    combinations = combinations.to_dict('records')
    if parameters in combinations:
        print("This combination of parameters has already been used")
        return (False, None)                                                       # return false if combination already used
    print(parameters)
    return (True, parameters)                                                              # true if combination not used yet 

def checkForWarmStarting(path, num1, num2):                                     # returns False if no warmstarting is possible
    df = pd.read_csv(path)
    array = [num2]
    df = df.loc[(df['num_neurons'] == num1) & df['num_neurons2'].isin(array)]
    if len(df) != 0:
        min_val = sorted(df["validation_loss"])                                # returns the one with the best validation loss
        exp = df.loc[df['validation_loss'] == min_val[0]]["experiment"]
        return exp.iloc[0]
    return False  

In [6]:
class trialRegressor(nn.Module):
    def __init__(self, num_neurons, num_neurons2, dropout):  
        super(trialRegressor, self).__init__()
        self.input_size = input_size
        self.num_neurons = num_neurons
        self.num_neurons2 = num_neurons2
        self.dropout = dropout
            
        self.fc1 = nn.Linear(self.input_size, self.num_neurons)
        self.fc2 = nn.Linear(self.num_neurons, self.num_neurons2)
        self.fc3 = nn.Linear(self.num_neurons2, 1)
        self.dropt = nn.Dropout(self.dropout)

    def forward(self, x):
        h = F.relu(self.fc1(x.float()))
        h1 = F.relu(self.fc2(self.dropt(h)))
        return self.fc3(self.dropt(h1))

In [7]:
def loss_function(real, predict, loss):
    if loss == "L1_loss":
        return F.l1_loss(predict, real)
    elif loss == "MSE_loss":
        return F.mse_loss(predict, real)
    else:
        return F.smooth_l1_loss(predict, real)

In [8]:
def optimizer_function(optimizer, model, lr):
    if optimizer == "Adam":
        return optim.Adam(model.parameters(), lr=lr)
    elif optimizer == "SGD":
        return optim.SGD(model.parameters(), lr=lr, weight_decay=0.1)
    elif optimizer == "RMSprop":
        return optim.RMSprop(model.parameters(), lr=lr)
    else:
        return optim.Adadelta(model.parameters(), lr=lr)    

In [9]:
def train(epoch, lo):
    model.train()
    train_loss = 0
    for _, (data, labels) in enumerate(train_loader):
        # print("Epoch: {}, Batch index: {}".format(epoch, batch_idx))
        data = data.to(device)
        labels = labels.to(device)
        optmz.zero_grad()
        output = model(data)
        loss = loss_function(labels.float(), output, lo)
        loss.backward()
        train_loss += loss.item()
        optmz.step()
    with torch.no_grad():
        loss_evolution.append(loss.cpu().detach().numpy())   

    print('====> Epoch: {} Average loss: {:.4f}'.format(          
        epoch, loss / len(train_loader.dataset)))

    # print('====> Epoch: {} Average loss: {:.4f}'.format(
    #      epoch, train_loss / len(train_loader.dataset)))


def validate(epoch, lo):
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for batch_idx, (data, labels) in enumerate(validation_loader):
            data = data.to(device)
            labels = labels.to(device)
            output = model(data)
            loss = loss_function(labels.float(), output, lo)
            valid_loss += loss.item()
            if epoch == parameters["epochs"]:
                REC_targets.append(labels.cpu().numpy().reshape(1, -1))             # take the whole dataset
                REC_predictions.append(output.cpu().reshape(1, -1))
        valid_loss_evolution.append(loss.cpu().numpy()) 
    # valid_loss /= len(validation_loader.dataset)                              
    # print('====> Validation set loss: {:.4f}'.format(valid_loss))

In [ ]:
for i in range(repetitions):
    while(True):
        flag, parameters = checkIdentical("path pointing to the directory where the file containing all the previously tried combination is \Records.csv", fillDict(parameters))
        if flag:                                                              # if combination never used exit
            break
    directory_path = "path where to save the results" + "\\" + str(datetime.datetime.now().
                                                                                     strftime("%d_%m_%Y-%H.%M"))
    print("Repetition: ", i + 1)
    model = trialRegressor(parameters["num_neurons"], parameters["num_neurons2"], parameters["dropout"])
    
                                    # check for a possible warm starting
    check = checkForWarmStarting("path \Pre_Records.csv", #  where the file containing all the previously tried combinations (different from the one above)  
                                 parameters["num_neurons"], parameters["num_neurons2"])# is stored, if you want to initialise your network with pre-copmputed weights
    
    if check :
        print("Warm starting from", check)
        model.load_state_dict(torch.load("path to the directory where the associated weights are stored" + "weights.pt"), 
                                         strict=True)
        
    model.to(device)
      
    optmz = optimizer_function(parameters["optimizer"], model, parameters["learning_rate"])
    step_size = parameters["epochs"]//2
    gamma = parameters["gamma"]
    scheduler = optim.lr_scheduler.StepLR(optmz, step_size=step_size, gamma=gamma)
    
    train_loader = torch.utils.data.DataLoader(training_set, shuffle=True, batch_size=parameters["batch_size"], **kwargs)
    validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=parameters["batch_size"], shuffle=True, **kwargs)
    
    REC_targets = list()
    REC_predictions = list()
    loss_evolution = list()
    valid_loss_evolution = list()
    
    # training
    for epoch in range(1, parameters["epochs"] + 1):                                   
        scheduler.step()
        train(epoch, parameters["loss"])
        validate(epoch, parameters["loss"])  
        
    # save the model    
    if not os.path.exists(directory_path):                
        os.makedirs(directory_path)
    
    #save the losses for comparison (superfluous given the saves in the csv file)
    text_file = open("where to save the losses \Losses.txt", "a")
    text_file.write("\n" + directory_path[66:] + "  " + str(valid_loss_evolution[-10:]))
    text_file.close()
    
    torch.save(model.state_dict(), directory_path + "\\weights.pt")   
    
    visualizeLossesOverEpochs(valid_loss_evolution, loss_evolution, 0,
                          "Validation loss", "Training loss", "0", 50, 100, 'o-', False)

    plt.savefig(fname=directory_path + "\\validation_loss.png", bbox_inches="tight")
    
    # save full validation loss
    text_file = open(directory_path +"\\validation_loss.txt", "w")
    text_file.write(str(valid_loss_evolution) + "\n")
    text_file.close()
    
    # save last batch targets
    text_file = open(directory_path +"\\REC_targets.txt", "w")
    text_file.write(str(REC_targets) + "\n")
    text_file.close()
    
    # save last batch predictions
    text_file = open(directory_path +"\\REC_predictions.txt", "w")
    text_file.write(str(REC_predictions) + "\n")
    text_file.close()
    
    # write the parameters in the csv file
    combinations = pd.read_csv("path \Records.csv") 
    if "Unnamed: 0" in combinations.columns:
        combinations.drop("Unnamed: 0", axis=1, inplace=True)                # change according to the structure of your file
    combinations = combinations.to_dict('records')
    parameters["training_loss"] = loss_evolution[-1]
    parameters["validation_loss"] = valid_loss_evolution[-1]
    parameters["training_path"] = str(training_path)
    parameters["test_path"] = str(test_path)
    parameters["experiment"] = directory_path[66:]
    combinations.append(parameters)                                               
    records = pd.DataFrame(combinations)
    records.to_csv("path \Records.csv", encoding="utf-8")
    
    text_file = open(directory_path + "\\" + "Parameters.txt", "w")
    text_file.write("Parameters used: \n\n")
    for _, (key, value) in enumerate(parameters.items()):
        text_file.write(key + " = " + str(value) + "\n")
    text_file.write("\n Dataset path: " + str(dataset_path))
    text_file.write("\n Training set path: " + str(training_path))  
    text_file.write("\n Test set path: " + str(test_path))
    
    text_file.close()
    
    plt.close("all")                                            # close all the figures in order to save memory